In [190]:
import pandas as pd
import json
import folium
import geopandas

In [303]:
coordinate_accuracy = 7;
coordinate_cluster_accuracy = 3;
jakob = 'wolfi4_2020-03-24t15_55_10.999z.json'
wolfi = 'wolfi4_2020-03-25t19_29_26.969z.json'
steffi = 'wolfi4_2020-03-24t16_32_43.640z.json'

In [386]:
with open("/home/woolf/projects/covid-19-tracing/geo-heatmap/datafiles/{}".format(jakob)) as f: 
    data = json.load(f)

In [387]:
locations = pandas.DataFrame.from_dict(data['locations']).filter(items=['timestampMs', 'latitudeE7', 'longitudeE7', 'accuracy'])
locations = locations.set_index(pandas.to_datetime(locations.timestampMs, unit='ms')) #TODO: Timezone!!!
locations = locations.sort_index()
locations['latitude'] = round(locations.latitudeE7 / 1e7, coordinate_accuracy)
locations['longitude'] = round(locations.longitudeE7 / 1e7, coordinate_accuracy)
locations['latitude_cluster'] = round(locations.latitudeE7 / 1e7, coordinate_cluster_accuracy)
locations['longitude_cluster'] = round(locations.longitudeE7 / 1e7, coordinate_cluster_accuracy)

# duration from one location to the next location
locations['duration'] = locations['timestampMs'].astype(int).diff(periods=1)

In [388]:
nightLocations = locations.between_time('23:00', '4:00')

groupedNightLocations = nightLocations\
    .groupby(['latitude_cluster', 'longitude_cluster'])\
    .agg({'duration': ['sum','count']})\
    .sort_values(by=('duration','sum'),ascending=0)
#topLocations = groupedNightLocations[groupedNightLocations > 2 ]
groupedNightLocations['minutes'] = groupedNightLocations[('duration','sum')]/60/1000

In [389]:
groupedNightLocations

duration            minutes
                                            sum count             
latitude_cluster longitude_cluster                                
47.257           11.372             248644360.0   153  4144.072667
47.404           10.889              33103986.0    19   551.733100
47.259           11.374              17837128.0     9   297.285467
47.258           11.375               2097803.0     1    34.963383

In [390]:
startpoint = groupedNightLocations.index[0]
map = folium.Map(startpoint, zoom_start=12, 
tiles='cartodbpositron', width=640, height=480)

[folium.CircleMarker((index[0],index[1]),
    radius=1,
    color='#0080bb',
    fill_color='#0080bb').add_to(map) for index, row in groupedNightLocations.iterrows()]

In [188]:
gdf = geopandas.GeoDataFrame(
    locations, geometry=geopandas.points_from_xy(locations.longitudeE7, locations.latitudeE7))

In [309]:
topLocations

latitude_cluster  longitude_cluster
47.257            11.372               153
47.404            10.889                20
47.259            11.374                 9
Name: count, dtype: int64

In [391]:
map

In [344]:
locations[(locations['latitude_cluster'] == 47.404) & (locations['longitude_cluster'] == 10.889)]

,timestampMs,latitudeE7,longitudeE7,accuracy,latitude,longitude,latitude_cluster,longitude_cluster
timestampMs,,,,,,,,
2020-02-14 23:31:41.438,1581723101438,474041331,108891216,16,47.404133,10.889122,47.404,10.889
2020-02-14 23:41:53.403,1581723713403,474041331,108891216,16,47.404133,10.889122,47.404,10.889
2020-02-14 23:56:52.777,1581724612777,474041331,108891216,16,47.404133,10.889122,47.404,10.889
2020-02-15 00:34:52.722,1581726892722,474041331,108891216,16,47.404133,10.889122,47.404,10.889
2020-02-15 01:12:13.264,1581729133264,474041331,108891216,16,47.404133,10.889122,47.404,10.889
...,...,...,...,...,...,...,...,...
2020-02-16 15:46:29.890,1581867989890,474040915,108890413,16,47.404091,10.889041,47.404,10.889
2020-02-16 15:48:40.172,1581868120172,474039594,108890898,28,47.403959,10.889090,47.404,10.889
2020-02-16 15:50:40.986,1581868240986,474041442,108887428,41,47.404144,10.888743,47.404,10.889
